# Survey Analysis

Including order summary and matching to payments.

In [ ]:
__author__ = "christineiym"

## Part 0: Setup

Import necessary packages.

In [ ]:
import pandas as pd
import numpy as np
import re

Define necessary constants.

In [ ]:
SURVEY_CSV_PATH: str = "survey_edited.csv"
VENMO_CSV_PATH: str = "venmo_statement_sept_oct_21.csv"
INDIVIDUAL_SUMMARY_CSV_PATH: str = "../indiv_summaries_all.csv"
ACCOUNTING_ALL_CSV_PATH: str = "../accounting_all_copy.csv"
ACCOUNTING_DIF_CSV_PATH: str = "../accounting_dif_copy.csv"

IGNORE_SURVEY_CSV_ROWS: list[int] = [0, 2]
IGNORE_VENMO_CSV_ROWS: list[int] = [0, 1, 3]
MAX_SHIRTS_IN_ORDER: int = 100
SHORT_PRICE: int = 15
LONG_PRICE: int = 20

SIZE_COLUMN_SUFFIX: str = " - Shirt [CurrentLoopNumber] size"
SLEEVE_LENGTH_COLUMN_SUFFIX: str = " - Shirt [CurrentLoopNumber] sleeve length"

ALL_SIZE_COLUMNS: list[str] = [(str(i) + SIZE_COLUMN_SUFFIX) for i in range(1, MAX_SHIRTS_IN_ORDER + 1)]
ALL_SLEEVE_LENGTH_COLUMNS: list[str] = [(str(i) + SLEEVE_LENGTH_COLUMN_SUFFIX) for i in range(1, MAX_SHIRTS_IN_ORDER + 1)]

Read in the Qualtrics survey CSV and confirm it was read in correctly.

In [ ]:
SURVEY_CSV_RAW: pd.DataFrame = pd.read_csv(SURVEY_CSV_PATH, encoding="utf-8", skiprows=IGNORE_SURVEY_CSV_ROWS)

# Drop columns with no information.
# SURVEY_CSV = SURVEY_CSV.dropna(axis=1, how='all')

SURVEY_CSV_RAW.head()

Create individual order summaries.

_Combine shirt sizes and sleeve lengths._

In [ ]:
SURVEY_CSV_INDIV = SURVEY_CSV_RAW.copy(deep=True)
SURVEY_CSV_INDIV = SURVEY_CSV_INDIV[ALL_SIZE_COLUMNS + ALL_SLEEVE_LENGTH_COLUMNS]

shirt_num: int = 1
while (shirt_num <= MAX_SHIRTS_IN_ORDER):
    current_shirt_col: str = 'Shirt ' + str(shirt_num)
    current_size_col: str = str(shirt_num) + SIZE_COLUMN_SUFFIX
    current_length_col: str = str(shirt_num) + SLEEVE_LENGTH_COLUMN_SUFFIX
    SURVEY_CSV_INDIV[current_shirt_col] = SURVEY_CSV_INDIV[current_length_col] + \
        " " + SURVEY_CSV_INDIV[current_size_col]
    SURVEY_CSV_INDIV = SURVEY_CSV_INDIV.drop(columns=[current_size_col, current_length_col])

    shirt_num += 1

# Drop columns with no information.
SURVEY_CSV_INDIV = SURVEY_CSV_INDIV.dropna(axis=1, how='all')

SURVEY_CSV_INDIV.head()

_Get the individual summaries._

In [ ]:
SURVEY_CSV_INDIV_SUM = SURVEY_CSV_INDIV.apply(pd.Series.value_counts, axis=1).fillna(0)
SURVEY_CSV_INDIV_SUM = SURVEY_CSV_INDIV_SUM.astype(int)
SURVEY_CSV_INDIV_SUM.columns

# To reorder columns; based off of seeing the column names
SURVEY_CSV_INDIV_SUM = SURVEY_CSV_INDIV_SUM[["Short S", "Short M", "Short L", "Short XL", "Short XXL", "Long XS", "Long S", "Long M", "Long L", "Long XL", "Long XXL"]]

SURVEY_CSV_INDIV_CONTACT = SURVEY_CSV_RAW[['First Name', 'Last Name', 'Email', 'Phone Number', 'Year/Status', 'Total Number of Shirts', 'Payment method', 'TotalPrice']]

SURVEY_CSV_INDIV_COMPLETE = SURVEY_CSV_INDIV_CONTACT.join(SURVEY_CSV_INDIV_SUM, how='outer')
SURVEY_CSV_INDIV_COMPLETE.head()

SURVEY_CSV_INDIV_COMPLETE.to_csv(INDIVIDUAL_SUMMARY_CSV_PATH)

Transform the raw CSV to make analysis for part 2 easier.

_Combine first and last names._

In [ ]:
SURVEY_CSV_FULL_NAMES = SURVEY_CSV_RAW.copy(deep=True)

SURVEY_CSV_FULL_NAMES["Full Name"] = SURVEY_CSV_FULL_NAMES["First Name"] + " " + SURVEY_CSV_FULL_NAMES["Last Name"]

SURVEY_CSV_FULL_NAMES.head()

_Transpose size columns._

In [ ]:
SURVEY_CSV_SIZES = SURVEY_CSV_FULL_NAMES.melt(id_vars=['Full Name', 'Payment method'], value_vars=ALL_SIZE_COLUMNS, value_name='Shirt Size')
SURVEY_CSV_SIZES = SURVEY_CSV_SIZES.drop(columns=['variable'])

SURVEY_CSV_SIZES.head()

_Transpose sleeve length columns._

In [ ]:
SURVEY_CSV_SLEEVE_LENGTH = SURVEY_CSV_FULL_NAMES.melt(id_vars=['Full Name', 'Payment method'], value_vars=ALL_SLEEVE_LENGTH_COLUMNS, \
    value_name='Shirt Sleeve Length')
SURVEY_CSV_SLEEVE_LENGTH = SURVEY_CSV_SLEEVE_LENGTH.drop(columns=['variable'])

SURVEY_CSV_SLEEVE_LENGTH.head()

_Join the size and sleeve length dataframes to obtain the final dataframe._

In [ ]:
SURVEY_CSV = SURVEY_CSV_SIZES.join(SURVEY_CSV_SLEEVE_LENGTH, how='outer', rsuffix='_duplicate')
SURVEY_CSV = SURVEY_CSV.drop(columns=['Payment method_duplicate', 'Full Name_duplicate'])

SURVEY_CSV.describe()

In [ ]:
SURVEY_CSV.to_csv(path_or_buf="test.csv")

Read in the Venmo CSV and confirm it was read in correctly.

In [ ]:
VENMO_CSV_RAW: pd.DataFrame = pd.read_csv(VENMO_CSV_PATH, header=None, encoding="utf-8", skiprows=IGNORE_VENMO_CSV_ROWS)

# Drop first column of dataframe
VENMO_CSV = VENMO_CSV_RAW.iloc[: , 1:]

# Set first row as header and delete first row
VENMO_CSV.columns = VENMO_CSV.iloc[0]
VENMO_CSV = VENMO_CSV.iloc[1: , :]

VENMO_CSV.head()

## Part 1: Summarize Orders

Obtain totals for each type of shirt and display results in a contingency table.

In [ ]:
data_crosstab: pd.DataFrame = pd.crosstab([SURVEY_CSV['Shirt Size']], SURVEY_CSV['Shirt Sleeve Length'], \
    margins = True, margins_name="Total")

print(data_crosstab)

Break down results by payment method.

In [ ]:
data_crosstab_detailed: pd.DataFrame = pd.crosstab([SURVEY_CSV['Payment method'], SURVEY_CSV['Shirt Size']], SURVEY_CSV['Shirt Sleeve Length'], \
    margins = True, margins_name="Total")

print(data_crosstab_detailed)

Obtain less detailed totals (by not breaking down by shirt size).

In [ ]:
data_crosstab_no_shirt_size: pd.DataFrame = pd.crosstab(SURVEY_CSV['Payment method'], SURVEY_CSV['Shirt Sleeve Length'], \
    margins = True, margins_name="Total")

print(data_crosstab_no_shirt_size)

Obtain raw totals expected from the fundraiser. 

In [ ]:
paypal_long_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Payment method'] == "PayPal") & (SURVEY_CSV['Shirt Sleeve Length'] == "Long"), \
    'Full Name'].count()
paypal_short_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Payment method'] == "PayPal") & (SURVEY_CSV['Shirt Sleeve Length'] == "Short"), \
    'Full Name'].count()
venmo_long_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Payment method'] == "Venmo") & (SURVEY_CSV['Shirt Sleeve Length'] == "Long"), \
    'Full Name'].count()
venmo_short_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Payment method'] == "Venmo") & (SURVEY_CSV['Shirt Sleeve Length'] == "Short"), \
    'Full Name'].count()
all_long_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Shirt Sleeve Length'] == "Long"), 'Full Name'].count()
all_short_count: int = SURVEY_CSV.loc[(SURVEY_CSV['Shirt Sleeve Length'] == "Short"), 'Full Name'].count()

print(f"Total $ expected in PayPal: {(paypal_long_count * LONG_PRICE) + (paypal_short_count * SHORT_PRICE)}")
print(f"Total $ expected in Venmo: {(venmo_long_count * LONG_PRICE) + (venmo_short_count * SHORT_PRICE)}")
print(f"\nTotal gross $ expected (regardless of payment): {(all_long_count * LONG_PRICE) + (all_short_count * SHORT_PRICE)}")

## Part 2: Account Balancing

Determine how much each person should pay.

In [ ]:
SURVEY_CSV_TOTALS = SURVEY_CSV_FULL_NAMES.copy(deep=True)
SURVEY_CSV_TOTALS = SURVEY_CSV_TOTALS[['Full Name', 'Email', 'Payment method', 'TotalPrice']]
SURVEY_CSV_TOTALS['Full Name'] = SURVEY_CSV_TOTALS['Full Name'].apply(lambda name: name.lower())
SURVEY_CSV_TOTALS['TotalPrice'] = SURVEY_CSV_TOTALS['TotalPrice'].apply(lambda amount: float(amount))

# Save email and payment method info for later.
SURVEY_CSV_TEXT_INFO = SURVEY_CSV_TOTALS[['Full Name', 'Email', 'Payment method']]

SURVEY_CSV_TOTALS.head()

Determine how much each person has paid.

In [ ]:
VENMO_CSV_TOTALS = VENMO_CSV.copy(deep=True)
VENMO_CSV_TOTALS = VENMO_CSV_TOTALS[['From', 'Note', 'Amount (total)']]
VENMO_CSV_TOTALS.rename(columns={'From':'Full Name', 'Amount (total)':'Amount paid'}, inplace=True)
VENMO_CSV_TOTALS['Full Name'] = VENMO_CSV_TOTALS['Full Name'].apply(lambda name: str(name).lower())
VENMO_CSV_TOTALS['Amount paid'] = VENMO_CSV_TOTALS['Amount paid'].apply(lambda raw_payment: re.sub("[^0-9|^/.|^-]", "", str(raw_payment)))
VENMO_CSV_TOTALS['Amount paid'] = VENMO_CSV_TOTALS['Amount paid'].apply(lambda payment: float(payment) if payment != '' else np.NaN)
# VENMO_CSV_TOTALS = VENMO_CSV_TOTALS.dropna()

VENMO_CSV_TEXT_INFO = VENMO_CSV_TOTALS[['Full Name', 'Note']]

VENMO_CSV_TOTALS.head()

Combine records of amounts owed with records of payments made and export the result to a csv.

In [ ]:
ACCOUNTING_INFO = SURVEY_CSV_TOTALS.merge(VENMO_CSV_TOTALS, how='outer', on='Full Name')
ACCOUNTING_INFO['TotalPrice'] = ACCOUNTING_INFO['TotalPrice'].fillna(0.0)
ACCOUNTING_INFO['Amount paid'] = ACCOUNTING_INFO['Amount paid'].fillna(0.0)
ACCOUNTING_INFO['Amount Owed'] = ACCOUNTING_INFO['TotalPrice'] - ACCOUNTING_INFO['Amount paid']
ACCOUNTING_INFO_CORRECTED = ACCOUNTING_INFO.groupby(["Full Name"]).sum()

# Add back email and note info.
ACCOUNTING_INFO_ADD_SURVEY_INFO = ACCOUNTING_INFO_CORRECTED.merge(SURVEY_CSV_TEXT_INFO, how='left', on='Full Name')
ACCOUNTING_INFO_COMPLETE = ACCOUNTING_INFO_ADD_SURVEY_INFO.merge(VENMO_CSV_TEXT_INFO, how='left', on='Full Name')

ACCOUNTING_INFO_COMPLETE.to_csv(path_or_buf=ACCOUNTING_ALL_CSV_PATH)
ACCOUNTING_INFO_COMPLETE.head()

Export rows where there is a difference in amount paid and amount owed to another csv.

In [ ]:
ACCOUNTING_INFO_DIFFERENCES = ACCOUNTING_INFO_COMPLETE[ACCOUNTING_INFO_COMPLETE['Amount Owed'] != 0.0]

ACCOUNTING_INFO_DIFFERENCES.to_csv(path_or_buf=ACCOUNTING_DIF_CSV_PATH)
ACCOUNTING_INFO_DIFFERENCES.head()